In [1]:
USE {
  dependencies {
    implementation("io.github.p-org.solvers:z3:4.8.14-v5")
  }
}

In [2]:
import java.io.File

data class Hailstone(
    val sx: Long,
    val sy: Long,
    val sz: Long,
    val vx: Long,
    val vy: Long,
    val vz: Long,
    val a: Long,
    val b: Long,
    val c: Long
) {
    constructor(
        sx: Long,
        sy: Long,
        sz: Long,
        vx: Long,
        vy: Long,
        vz: Long,
    ) : this(
        sx, sy, sz, vx, vy, vz, vy, -vx, (vy * sx) - (vx * sy)
    )
}

val hailstones = File("input.txt").readLines().map { line ->
    val values = line.replace("@", ",").split(",").map { it.trim(' ').toLong() }
    Hailstone(values[0], values[1], values[2], values[3], values[4], values[5])
}


hailstones.foldIndexed(0) { i, acc, hs1 ->
    acc + hailstones.subList(0, i).fold(0) { innerAcc, hs2 ->
        val a1 = hs1.a
        val b1 = hs1.b
        val c1 = hs1.c
        val a2 = hs2.a
        val b2 = hs2.b
        val c2 = hs2.c

        if (a1 * b2 == b1 * a2) return@fold innerAcc

        val x = (c1.toDouble() * b2 - c2.toDouble() * b1) / (a1.toDouble() * b2 - a2.toDouble() * b1)
        val y = (c2.toDouble() * a1 - c1.toDouble() * a2) / (a1.toDouble() * b2 - a2.toDouble() * b1)

        val xRange = 200000000000000.0..400000000000000.0
        val yRange = 200000000000000.0..400000000000000.0


        if (x in xRange && y in yRange && listOf(hs1, hs2)
                .all { (x - it.sx) * it.vx >= 0 && (y - it.sy) * it.vy >= 0 }
        ) {
            return@fold innerAcc + 1
        }
        innerAcc
    }
}

25433

## Pt 2 - using microsoft Z3 with java bindings to solve.
On Mac OS I had to use this [solution](https://github.com/Z3Prover/z3/issues/4979#issuecomment-773650678) to get it working.
Added following arguments to JVM -Djava.library.path="path/to/z3/bin"


In [9]:
operator fun <T> List<T>.component6(): T = get(5)

val hailstones = File("input.txt").readLines().map { line ->
    val values = line.replace("@", ",").split(",").map { it.trim(' ').toLong() }
    Hailstone(values[0], values[1], values[2], values[3], values[4], values[5])
}

lateinit var result: Expr<RealSort>
val ctx = Context()
val solver = ctx.mkSolver()

val (x, y, z, vx, vy, vz) = listOf("x", "y", "z", "vx", "vy", "vz").map { ctx.mkRealConst(it) }
(0..2).forEach { idx ->
    val h = hailstones[idx]
    val t = ctx.mkRealConst("t$idx")
    solver.add(
        ctx.mkEq(
            ctx.mkAdd(x, ctx.mkMul(vx, t)),
            ctx.mkAdd(ctx.mkReal(h.sx), ctx.mkMul(ctx.mkReal(h.vx), t))
        )
    )
    solver.add(
        ctx.mkEq(
            ctx.mkAdd(y, ctx.mkMul(vy, t)),
            ctx.mkAdd(ctx.mkReal(h.sy), ctx.mkMul(ctx.mkReal(h.vy), t))
        )
    )
    solver.add(
        ctx.mkEq(
            ctx.mkAdd(z, ctx.mkMul(vz, t)),
            ctx.mkAdd(ctx.mkReal(h.sz), ctx.mkMul(ctx.mkReal(h.vz), t))
        )
    )
}

if (solver.check() == Status.SATISFIABLE) {
    result = solver.model.eval(ctx.mkAdd(x, ctx.mkAdd(y, z)), false)
}
result

885093461440405